In [ ]:
!pip install transliterate -q

In [ ]:
!pip install nltk -q

In [ ]:
!pip install -U -q sentence-transformers

# Matching товаров ООО "ПРОСЕПТ"

## Введение

**ООО «ПРОСЕПТ»** — российская производственная компания, специализирующаяся
на выпуске профессиональной химии. В своей работе используют опыт ведущих
мировых производителей и сырье крупнейших химических концернов. Производство и
логистический центр расположены в непосредственной близости от Санкт-Петербурга,
откуда продукция компании поставляется во все регионы России.
Сайт: https://prosept.ru/


**Введение в задачу**:

Заказчик производит несколько сотен различных товаров бытовой и промышленной
химии, а затем продаёт эти товары через дилеров. Дилеры, в свою очередь,
занимаются розничной продажей товаров в крупных сетях магазинов и на онлайн
площадках.

Для оценки ситуации, управления ценами и бизнесом в целом, заказчик
периодически собирает информацию о том, как дилеры продают их товар. Для этого
они парсят сайты дилеров, а затем сопоставляют товары и цены.
Зачастую описание товаров на сайтах дилеров отличаются от того описания, что даёт
заказчик. Например, могут добавляться новый слова (“универсальный”,
“эффективный”), объём (0.6 л -> 600 мл). Поэтому сопоставление товаров дилеров с
товарами производителя делается вручную.
Цель этого проекта - разработка решения, которое отчасти автоматизирует процесс
сопоставления товаров. Основная идея - предлагать несколько товаров заказчика,
которые с наибольшей вероятностью соответствуют размечаемому товару дилера.
Предлагается реализовать это решение, как онлайн сервис, открываемый в веб-
браузере. Выбор наиболее вероятных подсказок делается методами машинного
обучения.

**Документация к предоставленным данным**:

Заказчик предоставил несколько таблиц (дамп БД), содержащих необходимые
данные:

1 marketing_dealer - список дилеров;

2 marketing_dealerprice - результат работы парсера площадок дилеров:

- product_key - уникальный номер позиции;

- price - цена;

- product_url - адрес страницы, откуда собраны данные;

- product_name - заголовок продаваемого товара;

- date - дата получения информации;

- dealer_id - идентификатор дилера (внешний ключ к marketing_dealer)


3 marketing_product - список товаров, которые производит и распространяет
заказчик;

- article - артикул товара;

- ean_13 - код товара (см. EAN 13)

- name - название товара;

- cost - стоимость;

- min_recommended_price - рекомендованная минимальная цена;

- recommended_price - рекомендованная цена;

- category_id - категория товара;

- ozon_name - названиет товара на Озоне;

- name_1c - название товара в 1C;

- wb_name - название товара на Wildberries;

- ozon_article - описание для Озон;

- wb_article - артикул для Wildberries;

- ym_article - артикул для Яндекс.Маркета;

4 marketing_productdealerkey - таблица матчинга товаров заказчика и товаров
дилеров

- key - внешний ключ к marketing_dealerprice

- product_id - внешний ключ к marketing_product

- dealer_id - внешний ключ к marketing_dealer

## План работ


**До дедлайна 19:00 29 ноября**

1. Команда знакомится с предоставленными данными.

2. Формулируется DS задача, утверждается единые схема валидации решений и метрика качества.

3. Выбирается основной способ решения задачи (модель первого этапа с функционалом финального решения),
    который будет представлен к дедлайну 29 ноября:
    - Рассматриваются и валидируются разные способы предобработки входных данных модели.
    - Рассматриваются и валидируются разные ml движки решения.


4. Подготовка модели первого этапа к сдаче на ревью в том виде, в котором ею сможет пользоваться BackEnd департамент команды.

5. Подготовка репозитория решения с jupyter notebook, содержащим основные вехи разработки модели первого этапа.

**После дедлайна DS до единого дедлайна**

6. Генерация новых фичей, улучшение схемы предобработки входных данных.

7. Построение и валидация модели второго этапа (реранжирующий классификатор).

8. Тюнинг реранжирующего классификатора.

9. Предоставление финального решения BackEnd команде, помощь в его инициализации.

**10**. Оформление документации и ожидание результатов хакатона.


## Решение (молель первого этапа)

### Импорты

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import KFold

import pandas as pd
import numpy as np
import seaborn as sns

from tqdm import tqdm

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from transliterate import translit
import re


from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel

import torch
import torch.nn.functional as F
from torch import Tensor

### Загрузка данных

In [ ]:
!ls -lah

In [ ]:
try:
    dealer = pd.read_csv('/kaggle/input/privat-matching/data/marketing_dealer.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    dealerprice = pd.read_csv('/kaggle/input/privat-matching/data/marketing_dealerprice.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    product = pd.read_csv('/kaggle/input/privat-matching/data/marketing_product.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    interactions = pd.read_csv('/kaggle/input/privat-matching/data/marketing_productdealerkey.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
except:
    dealer = pd.read_csv('marketing_dealer.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    dealerprice = pd.read_csv('marketing_dealerprice.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    product = pd.read_csv('marketing_product.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    interactions = pd.read_csv('marketing_productdealerkey.csv', on_bad_lines="skip", encoding='utf-8', sep=';')
    

#### Роль алгоритма в функционале приложения

Наш алгоритм должен помочь разметчику сопоставить товар диллера с одним из нескольких сотен товаров фирмы заказчика. Важно отметить, что финальное решение принимает именно разметчик. Так вот, насколько видит наша команда, алгоритм для каждого из предложенных товаров диллеров должен вернуть ранжированный список всех товаров заказчика так, чтобы релевантный айтем оказался максимально высоко в топе. Таким образом перед нами тривиальная задача ранжирования.

#### Особенности задачи

- Для товара диллера у заказчика есть только один релевантный айтем.
- Основными признаками для матчинга выступают нейминги товаров.
- Товары диллеров всегда новые, а множество товаров заказчика меняется редко.

#### Метрика качества

В качестве метрики качества мы утвердили Среднеобратный ранг (Mean Reciprocal Rank): $MRR = \frac{1}{N} \sum_{i=1}^{N} \frac{1}{\text{rank}_i}$ ,
где ${\text{rank}_i}$ это позиция релевантного айтема заказчика в ранжированном списке а ${N}$ это мощность множества товаров дилеров. Метрика выбрана по ряду причин, и по нашему общему мнению идеально совпадает с нашей задачей. MRR это ничто иное как средняя позиция правильного ответа (совпадения / релевантного item), для удобства шкалированная от 0 до 1. Чтобы из MRR получить среднюю позицию правильного ответа, достаточно делить на неё единицу. В случае одного релевантного товара nDCG (популярная метрика ранжирования) и MRR будут равны с точностью до константы.

#### Схема валидации

Т.к. соответствия товаров диллеров товарам заказчика не изменяются со временем, обычная кросс-валидация идеально нам подходит. Будем валидироваться на 5 фолдах, в тестовой выборке каждого фолда будет примерно 300-400 соответствий.

#### Признаки айтемов 
**Первый этап**:
Как ранее указывалось, основным признаком для сопоставления являются нейминги товаров. Таким образом в качестве модели первого этапа мы представим функцию, помещающую все нейминги товаров заказчика в единое векторное пространство с помощью текстового векторизатора. Для формирования списка рекомендаций нейминг товара дилера помещается в то же пространство, а затем товары заказчика ранжируются по косинусной близости. В итоге уже обученная модель будет принимать массив с неймингами товаров дилера, а возвращать двумерный массив с рекомендациями.

**Второй этап**:
В качестве признаков модель второго этапа (классификатор) для каждой пары `товар_дилера - товар_заказчика` будет принимать вектора неймингов этих товаров, косинусное расстояние между векторами, а так же другие features какие мы нагенерируем. Далее по предсказанным вероятностям соответствия все айтемы заказчика реранжируются.

### Знакомство с данными

**Таблица с результатами парсинга**

In [ ]:
dealerprice

In [ ]:
dealerprice.info()

In [ ]:
dealerprice.head()

Почему-то product_key хранится как строки, хотя должен храниться как числа. Проблема какая-то.

**p.s.** Некоторые диллеры хранят ключи своих товаров как ссылки 

In [ ]:
dealerprice.isna().sum()

In [ ]:
print(f"Уникальных диллеров в данных парсера: {len(dealerprice['dealer_id'].unique())}")

In [ ]:
prosept_count = dealerprice['product_name'].apply(lambda x: 1 if x.lower().find('prosept') != -1 else 0).copy().rename('prosept_count')
print(f"Название компании Prosept встречается в {prosept_count.sum()} названиях товаров ({prosept_count.mean() * 100:.2f}%)")

**Таблица с информацией о продуктах заказчиков**

In [ ]:
product.head(3)

In [ ]:
product.isna().sum()

In [ ]:
product.duplicated().sum()

**Таблица с сопоставлением продуктов дилеров продуктам заказчиков**

In [ ]:
interactions.info()

In [ ]:
interactions.head(3)

In [ ]:
interactions.isna().sum()

In [ ]:
interactions.duplicated().sum()

#### Предобработка грубых недостатков таблиц

*ссылки в ключах диллера*

**p.s.**  оказалось что не надо удалять

In [ ]:
# rows_indexes = dealerprice[dealerprice['product_key'].apply(lambda x: not x.strip().isdigit())].index
# dealerprice = dealerprice.drop(rows_indexes)
# del rows_indexes

*ссылки в ключах взаимодействий*

**p.s.**  оказалось что не надо удалять

In [ ]:
# rows_indexes = interactions[interactions['key'].apply(lambda x: not x.strip().isdigit())].index
# interactions = interactions.drop(rows_indexes)
# del rows_indexes

In [ ]:
# interactions = interactions.reset_index(drop=True)

*пропуски в title PRODUCT* 

In [ ]:
product[product['name'].isna()]

In [ ]:
product = product.dropna(subset=['name', 'recommended_price'])

In [ ]:
product = product.reset_index(drop=True)

*дубли в id диллеров*

**p.s.** Ключи продуктов уникальные лишь в рамках отдельного диллера, так что удаляем дубликаты по двум столбцам. Чтобы в датасете остались только актуальные записи, перед удалением дублей сортируем таблицу по дате.

In [ ]:
dealerprice = dealerprice.sort_values('date', ascending=False).drop_duplicates(subset=['product_key', 'dealer_id'])

In [ ]:
dealerprice = dealerprice.reset_index(drop=True)

#### Функции чтения и первичной обработки входящих сырых таблиц

Функция чтения  **marketing_dealerprice**:

In [ ]:
def dealerprice_table(table_path='marketing_dealerprice.csv',
                      product_id_column='product_key',
                      dealer_id_column='dealer_id',
                      read_params={'on_bad_lines': "skip",
                                   'encoding': 'utf-8',
                                   'sep': ';'}
                     ):
    '''
    Функция принимает:
    .Путь к csv файлу, содержащему результаты парсинга.
    .Названия колонок с id товаров и id дилеров
    .Параметры чтения csv можно указать, если вдруг они изменятся.
    '''
    
    table_csv = pd.read_csv(table_path, **read_params) 
    table_csv = table_csv.sort_values('date', ascending=False).drop_duplicates(subset=[product_id_column, dealer_id_column])
    
    return table_csv
    

Функция для чтения **marketing_product**

In [ ]:
def prossept_products_table(table_path='marketing_product.csv',
                            product_names_column = 'name',
                            read_params={'on_bad_lines': "skip",
                                           'encoding': 'utf-8',
                                           'sep': ';'}
                             ):
    '''
    Функция принимает путь к csv файлу, содержащему актуальную информацию по товарам заказчика.
    Дополнительно указывается название колонки с внутренними неймингами для удаления плохих строк.
    '''
    
    table_csv = pd.read_csv(table_path, **read_params) 
    table_csv = table_csv.dropna(subset='name')
    
    return table_csv

**Текущие функции подвергнутся изменениям и улучшениям в процессе инициализации решения с бекендом в прод.**

### Формулирование DS задачи

## Моделирование

Заглушка для бека

In [ ]:
class PopularRecommender():

    def __init__(self, ):
        pass

    def fit(self,
            interactions,
            product_id='product_id'):
        
        self.recs = interactions[product_id].value_counts().index.tolist()

    def recommend(self,
                  dealer_ids: list[dict]):
        
        return np.array([self.recs for i in dealer_ids])
    
model = PopularRecommender()
model.fit(interactions)
model.recommend([1, 2, 3])

### Модель первого этапа

Основным движком матчинга будет модель векторизации неймингов. На следующем этапе мы подтянем другие фичи для сопоставления. Например цены товаров. 

*функция предобработки неймингов для простых векторайзеров:*

In [ ]:
ru_stop = stopwords.words('russian')
eng_stop = stopwords.words('english')
ru_stemmer = SnowballStemmer("russian")
eng_stemmer = SnowballStemmer("english")

def string_filter(string,
                  ru_stop=ru_stop,
                  eng_stop=eng_stop,
                  ru_stemmer=ru_stemmer,
                  eng_stemmer=eng_stemmer):
    
    string = string.lower() 
    string = re.sub(r'[^a-zo0-9а-я\s:]', '', string)
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    string = ' '.join([eng_stemmer.stem(ru_stemmer.stem(word)) for word in string.split() if word not in ru_stop+eng_stop])

    return string

In [ ]:
class DistanceRecommender():

    def __init__(self,
                 vectorizer,
                 simularity_func,
                 text_prep_func):
        

        
        self.vectorizer = vectorizer
        self.simularity_counter = simularity_func
        self.preprocessing = text_prep_func

    def fit(self,
            product_corpus,
            name_column,
            id_column):
        preprocessed_corpus = product_corpus[name_column].apply(self.preprocessing).values.tolist()
        
        self.vectorizer.fit(preprocessed_corpus)
        self.product_matrix = self.vectorizer.transform(preprocessed_corpus)
        self.product_index_to_id = {i: product_corpus.loc[i, id_column] for i in range(len(product_corpus))}
        
    def recommend(self,
                  dealer_corpus: list[dict]
                 ):
    
        preprocessed_corpus = dealer_corpus.apply(self.preprocessing).values.tolist()
        vectors = self.vectorizer.transform(preprocessed_corpus)
        sims = self.simularity_counter(vectors, self.product_matrix)
        
        result = []
        for vec in sims:
            result += [[self.product_index_to_id[index] for index in vec.argsort()[::-1]]]
        return np.array(result)

Метрика MRR@10, для удобства своя реализация метрики Mean Reciprocal Rank ( https://rectools.readthedocs.io/en/latest/api/rectools.metrics.ranking.MRR.html ). Метрика идеально ложится на нашу задачу. Скалируется от 0 до 1, чем больше метрика тем выше релевантный айтем в списке рекоммендаций. 

In [ ]:
def accuracy_k(true_ids, recommendations, k):
    results = 0
    for index, true_id in enumerate(true_ids):
        if true_id in recommendations[index][:k]:
            results += 1
    return results / len(true_ids)

In [ ]:
def mean_reciprocal_rank(true_id,
                         recommendations,
                         k=10):
    
    reciprocal_ranks = []
    
    for i, rec in enumerate(recommendations):
        recs = rec[:k]
        relevant = true_id[i]
        
        if np.isin(relevant, recs):
            rank = np.where(recs == relevant)[0][0] + 1
            reciprocal_ranks += [1 / rank]
            
        else:
            reciprocal_ranks += [0]
            
    return np.mean(reciprocal_ranks)

In [ ]:
models = {
    'CosRecBow_1n': DistanceRecommender(vectorizer=CountVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecBow_1n_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecBow_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecBow_1n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecBow_2n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecBow_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_1n': DistanceRecommender(vectorizer=TfidfVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_1n_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_1n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_2n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter),
    'CosRecTfIDF_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter)
}

Все модели первого этапа обучаются на корпусе неймингов заказчика:

In [ ]:
for model in models.values():
    model.fit(product, 'name', 'id')

**кросс-валидация**

Валидировать решения будем на обычной кросс-валидации с 5 фолдами, ту же схему валидации будем использовать и для модели второго этапа. Все результаты будут сохраняться в единый фрейм:

In [ ]:
dealerprice

In [ ]:
kf = KFold(n_splits=5)

In [ ]:
results_table = pd.DataFrame()

for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name + '_string_filter'])])

In [ ]:
results_table.sort_values('MRR', ascending=False)

In [ ]:
del models

Лучше всего срабатывает поиск ближайших по косиносному расстоянию на TFIDF с униграммами. Попробуем улучишить функцию предобработки неймингов так, чтобы она приводила литры в миллилитры и килограммы в граммы:

In [ ]:
def replace_values_l(value):
    if ' л' in value:
        value = value.replace(' л', '000 мл')
        value = value.replace('.0', '')  
        return value
    elif 'л' in value:
        pattern = r'(\d+(?:\.\d+)?)\s*л\b'  
        matches = re.findall(pattern, value, flags=re.IGNORECASE)
        for match in matches:
            replacement = f"{float(match) * 1000:.0f} мл"  
            value = re.sub(fr'({match})\s*л\b', replacement, value, flags=re.IGNORECASE)
        return value
    else:
        return value

def replace_values_kg(value):
    if ' кг' in value:
        value = value.replace(' кг', '000 г')
        value = value.replace('.0', '')  
        return value
    elif 'кг' in value:
        pattern = r'(\d+(?:\.\d+)?)\s*кг\b' 
        matches = re.findall(pattern, value, flags=re.IGNORECASE)
        for match in matches:
            replacement = f"{float(match) * 1000:.0f} г"  
            value = re.sub(fr'({match})\s*кг\b', replacement, value, flags=re.IGNORECASE)
        return value
    else:
        return value

def string_filter_v2(string,
                  ru_stop=ru_stop,
                  eng_stop=eng_stop,
                  ru_stemmer=ru_stemmer,
                  eng_stemmer=eng_stemmer):
    
    string = string.lower() 
    string = replace_values_kg(replace_values_l(string))
    string = re.sub(r'[^a-zo0-9а-я\s:]', '', string)
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    string = ' '.join([eng_stemmer.stem(ru_stemmer.stem(word)) for word in string.split() if word not in ru_stop+eng_stop])

    return string

In [ ]:
models = {
    'CosRecBow_1n': DistanceRecommender(vectorizer=CountVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecBow_1n_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecBow_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecBow_1n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecBow_2n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecBow_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_1n': DistanceRecommender(vectorizer=TfidfVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_1n_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_1n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_2n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2),
    'CosRecTfIDF_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2)
}

In [ ]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name + 'filter_v2'])])

In [ ]:
results_table.sort_values('MRR', ascending=False)

Метрика подскочила, это хорошо. Теперь лидирует мешок слов со стандартными весами. Попробуем разбивать слова текста на пары или тройки букв и строить мешки из них. Разумеется, надо увеличивать количество n-грамм в мешках: 

In [ ]:
def generate_ngrams(text, n):
    ngrams = [text[i:i + n] for i in range(len(text) - n + 1)]
    return ngrams

def string_filter_v3(string,
                  ru_stop=ru_stop,
                  eng_stop=eng_stop,
                  ru_stemmer=ru_stemmer,
                  eng_stemmer=eng_stemmer,
                  ngram_len=2):
    
    string = string.lower() 
    string = replace_values_kg(replace_values_l(string))
    string = re.sub(r'[^a-zo0-9а-я\s:]', '', string)
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    string = ' '.join([eng_stemmer.stem(ru_stemmer.stem(word)) for word in string.split() if word not in ru_stop+eng_stop])
    string = ' '.join([' '.join(generate_ngrams(word, ngram_len)) if (len(word) >= ngram_len and not word.isdigit()) else word for word in string.split()])
    return string

string = dealerprice['product_name'][0]
print(string)
print(string_filter_v3(string))

In [ ]:
models = {
    'CosRecBow_1n': DistanceRecommender(vectorizer=CountVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_1n_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_1n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_2n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_1n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_2n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_3n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecBow_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(4,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_1n': DistanceRecommender(vectorizer=TfidfVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_1n_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_1n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_2n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_1n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_2n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_3n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
    'CosRecTfIDF_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(4,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3),
}

In [ ]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name + 'filter_v3'])])

In [ ]:
results_table.sort_values('MRR', ascending=False)

Результаты не улучшились. Попробуем добавить в предобработку транслитизацию. Сначала затестим без разбиения на пары символов:

In [ ]:
def string_filter_v2_t(string,
                  ru_stop=ru_stop,
                  eng_stop=eng_stop,
                  ru_stemmer=ru_stemmer,
                  eng_stemmer=eng_stemmer,
                  transliterator=translit):
    
    string = string.lower() 
    string = replace_values_kg(replace_values_l(string))
    string = re.sub(r'[^a-zo0-9а-я\s:]', '', string)
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    string = ' '.join([eng_stemmer.stem(ru_stemmer.stem(word)) for word in string.split() if word not in ru_stop+eng_stop])

    string = transliterator(string, 'ru', reversed=False)
    return string

In [ ]:
models = {
    'CosRecBow_1n': DistanceRecommender(vectorizer=CountVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecBow_1n_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecBow_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecBow_1n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecBow_2n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecBow_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_1n': DistanceRecommender(vectorizer=TfidfVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_1n_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_1n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_2n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t),
    'CosRecTfIDF_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v2_t)
}

In [ ]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name + '_filter_v2_t'])])

In [ ]:
results_table.sort_values('MRR', ascending=False)

Результаты улучшились Теперь с транслитизацие и разбиением на пары символов:

In [ ]:
def string_filter_v3_t(string,
                  ru_stop=ru_stop,
                  eng_stop=eng_stop,
                  ru_stemmer=ru_stemmer,
                  eng_stemmer=eng_stemmer,
                  ngram_len=2,
                  transliterator=translit):
    
    string = string.lower() 
    string = replace_values_kg(replace_values_l(string))
    string = re.sub(r'[^a-zo0-9а-я\s:]', '', string)
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    string = ' '.join([eng_stemmer.stem(ru_stemmer.stem(word)) for word in string.split() if word not in ru_stop+eng_stop])
    string = ' '.join([' '.join(generate_ngrams(word, ngram_len)) if (len(word) >= ngram_len and not word.isdigit()) else word for word in string.split()])
    string = transliterator(string, 'ru')
    return string

In [ ]:
models = {
    'CosRecBow_1n': DistanceRecommender(vectorizer=CountVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_1n_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_2n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_1n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_2n_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_3n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_1n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(1,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_2n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(2,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_3n_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(3,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecBow_4n': DistanceRecommender(vectorizer=CountVectorizer(ngram_range=(4,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_1n': DistanceRecommender(vectorizer=TfidfVectorizer(), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_1n_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_2n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,2)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_1n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_2n_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_3n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,3)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_1n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(1,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_2n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(2,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_3n_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(3,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
    'CosRecTfIDF_4n': DistanceRecommender(vectorizer=TfidfVectorizer(ngram_range=(4,4)), simularity_func=cosine_similarity, text_prep_func=string_filter_v3_t),
}

In [ ]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name + '_filter_v3_t'])])

In [ ]:
results_table.sort_values('MRR', ascending=False)

Транслитирация немного улучшает результаты, а разбиение токенов на пары символов - наоборот. Похоже что мы выжали максимум из мешков слов, а значит двигаемся к плотным векторным представлениям:

In [ ]:
def string_filter_emb(string):
    
    string = string.lower() 
    string = replace_values_kg(replace_values_l(string))
    string = re.sub(r'[^a-zo0-9а-я\s:]', ' ', string)
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    return string

string = dealerprice['product_name'][0]
print(string)
print(string_filter_emb(string))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_nlu_ru")
bert = AutoModel.from_pretrained("ai-forever/sbert_large_nlu_ru")

Оборачиваем sbert, чтобы работал как обычный векторайзер sklearn (чтобы корректно встал в DistanceRecommender). Далее аналогичные действия для других моделей не комментируются ^^

In [ ]:
class SbertVectorizer():
    
    def __init__(self,
                 tokenizer=tokenizer,
                 model=bert):
        
        self.tokenizer = tokenizer
        self.model = model
    
    def fit(self, X=None):        
        pass
    

    
    def transform(self, corpus):
        encoded_input = tokenizer(corpus, padding=True, truncation=True, max_length=24, return_tensors='pt')
        
        with torch.no_grad():
            model_output = self.model(**encoded_input) 
        
        token_embeddings = model_output[0]
        input_mask_expanded = encoded_input['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
        
        sentence_embeddings = sum_embeddings / sum_mask
        
        return sentence_embeddings.numpy()

In [ ]:
models = {'CosEmb_sbert': DistanceRecommender(vectorizer=SbertVectorizer(tokenizer=tokenizer, model=bert), simularity_func=cosine_similarity, text_prep_func=string_filter_emb)}

In [ ]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name])])

In [ ]:
results_table.loc['CosEmb_sbert', :]

In [ ]:
del tokenizer, bert

Наблюдаем низкий MRR, а это означает что модель нам не подходит. Вероятно проблема в том, что sbert корректно работает только с русскими токенами, а у нас в неймингах присутствуют английские.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
bert = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

In [ ]:
class BertVectorizer():
    
    def __init__(self,
                 tokenizer=tokenizer,
                 model=bert):
        
        self.tokenizer = tokenizer
        self.model = model
    
    def fit(self, X=None):        
        pass
    

    
    def transform(self, text):
        t = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**{k: v.to(self.model.device) for k, v in t.items()})
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        return embeddings.numpy()

In [ ]:
models = {'RubertTiny2_Cos': DistanceRecommender(vectorizer=BertVectorizer(tokenizer=tokenizer, model=bert), simularity_func=cosine_similarity, text_prep_func=string_filter_emb,)}

In [ ]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name])])

In [ ]:
results_table.loc[models.keys()]

In [ ]:
del tokenizer, bert

Наконец-то более-менее конкурентноспособные результаты от bert-like модели. Посмотрим реализацию sbert_large_mt_nlu_ru:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")
bert = AutoModel.from_pretrained("ai-forever/sbert_large_mt_nlu_ru")

In [ ]:
models = {'CosEmb_mt_sbert': DistanceRecommender(vectorizer=SbertVectorizer(tokenizer=tokenizer, model=bert), simularity_func=cosine_similarity, text_prep_func=string_filter_emb)}

In [ ]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name])])

In [ ]:
results_table.loc[models.keys()]

In [ ]:
del bert, tokenizer

No comments, двигаемся дальше. Теперь можно посмотреть берты работающие с несколькими языками, одним из лучших считаются LaBSE. Для начала глянем урезанную LaBSE, работающую только с русскими и английскими токенами:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
bert = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

In [ ]:
models = {'CosEmb_LaBSE-en-ru': DistanceRecommender(vectorizer=BertVectorizer(tokenizer=tokenizer, model=bert), simularity_func=cosine_similarity, text_prep_func=string_filter_emb)}

In [ ]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name])])

In [ ]:
results_table.loc[models.keys()]

In [ ]:
del bert, tokenizer

Метрика лучше чем у предыдущих бертов. Теперь посмотрим большую Labse. Приятно, что у неё есть обёртка SentenceTransformer.

In [ ]:
transformer = SentenceTransformer('sentence-transformers/LaBSE')

In [ ]:
class TransformerVectorizer():
    
    def __init__(self,
                 transformer=transformer):
        
        self.transformer = transformer
    
    def fit(self, X=None):        
        pass
    
    def transform(self, corpus):
        embeddings = transformer.encode(corpus) 
        return embeddings

In [ ]:
models = {'CosEmb_LaBSE': DistanceRecommender(vectorizer=TransformerVectorizer(transformer=transformer), simularity_func=cosine_similarity, text_prep_func=string_filter_emb)}

In [ ]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name])])

In [ ]:
results_table.loc[models.keys()]

In [ ]:
del transformer

Продвинулись дальше, но пока не догнали мешки с n-граммами. Напоследок посмотрим state of the art векторайзер на русских текстах:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
vectorizer = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

class InfloatVectorizer():
    
    def __init__(self,
                 tokenizer=tokenizer,
                 vectorizer=vectorizer):
        
        self.tokenizer = tokenizer
        self.model = vectorizer
    
    def fit(self, X=None):        
        pass
    

    
    def transform(self, corpus):
        batch_dict = self.tokenizer(corpus, max_length=512, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            outputs = self.model(**batch_dict)
        last_hidden = outputs.last_hidden_state.masked_fill(~batch_dict['attention_mask'][..., None].bool(), 0.0)
        embeddings = last_hidden.sum(dim=1) / batch_dict['attention_mask'].sum(dim=1)[..., None]
        embeddings = F.normalize(embeddings, p=2, dim=1)
        return embeddings.numpy()

In [ ]:
models = {'Infloat_multilingual': DistanceRecommender(vectorizer=InfloatVectorizer(tokenizer=tokenizer, vectorizer=vectorizer), simularity_func=cosine_similarity, text_prep_func=string_filter_emb)}

In [ ]:
for model in models.values():
    model.fit(product, 'name', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name])])

In [ ]:
results_table.sort_values('MRR', ascending=False)

Наконец-то удалось победить мешок с n-граммами. Хочется отметить, что помимо показанных выше bert-like моделей командой были рассмотрены и отвалидированы несколько других. К сожалению они показали совсем плохие результаты, и тут мы их не смотрим. Ради инетереса (больше для фана) посмотрим самую популярную метрику хакатона - accuracy@k.

In [ ]:
del tokenizer, vectorizer

#### Результаты рассмотренных решений

In [ ]:
results_table.sort_values('MRR', ascending=False).head(20)

#### Выводы:

Лучший MRR показала модель, ранжирующая по косинусному сходству эмбеддинги xlm-roberta-large модели, предобученной командой *intfloat* (https://huggingface.co/intfloat/multilingual-e5-large). Текущее решение уже показывает отличные результаты, и вполне способно стать финальным. 
На кросс-валидации модель демонстрирует **0.811** **MRR@10**. Это означает, что **средний ранг** релевантного айтема в списке рекомендаций модели приблизительно 
равен **1.23** (чаще всего на первом месте, редко на втором, редко-редко на других). Таким образом разметчик сможет практически моментально находить релевантный айтем заказчика.

#### Отдаём backend команде:

Набор функций и методов можно найти в `main.py`. При первой инициализации в текущей директории подгружаются все файлы, необходимые для её работы. Предусмотрена возможность переобучить на новой базе данных заказчика. Для этого нужно вызвать метод `fit` модели на датасете содержащем информацию с ключами и неймингами товаров заказчика.

### Улучшение результата после дедлайна

#### Почему отказались от идеи реранжирования классификатором

Попытка реранжировать результаты работы модели первого этапа классификатором увенчалась большим провалом. Мы проверили очень много разных способов обучать классификатор catboost на аутпутах модели первого этапа, и лучший результат которого нам удалось достичь это **0.4 MRR**. Насколько мы видим основная проблема подхода в малом количестве положительных примеров матчинга. Это скажем мягко говоря sad. К слову, так же пробовали обогащать вектора мешков с n-граммами разными фичами и так же ранжировать по расстоянию. Это оказался менее неудачный опыт, но всё же неудачный. Из всего заключаем, что первое решение теперь финальное. 

Добавим небольшое улучшение, теперь вектора товаров заказчика получаем из объединённых неймингов из колонок `name` и `ozon_name` если есть: 

In [ ]:
def names_join_ozon(x):
    total = []
    if type(x['name']) == str:
        total += [x['name'].strip()]
    if type(x['ozon_name']) == str:
        total += [x['ozon_name'].strip()]
    return ' '.join(total)

In [ ]:
product['name_with_ozon'] = product.apply(names_join_ozon, axis=1)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
vectorizer = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

In [ ]:
models = {'Infloat_multilingual_with_ozon': DistanceRecommender(vectorizer=InfloatVectorizer(tokenizer=tokenizer, vectorizer=vectorizer), simularity_func=cosine_similarity, text_prep_func=filter_emb)}

In [ ]:
for model in models.values():
    model.fit(product, 'name_with_ozon', 'id')

In [ ]:
results_table = pd.DataFrame()

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name])])

In [ ]:
results_table

посмотрим как можно улучшить чистку неймингов:

In [ ]:
def replace_values_l1(string):
    # Найти числа и единицы измерения в строке
    # Найти числа с единицами измерения в строке
    matches = re.findall(r"(\d+(?:[\.,]\d+)?)\s*(л|литр)", string, flags=re.IGNORECASE)
    
    # Заменить литры на миллилитры
    for match in matches:
        value = float(match[0].replace(",", ".")) * 1000
        replacement = f"{int(value)} мл"
        string = re.sub(fr"\b{match[0]}\s*{re.escape(match[1])}\b", replacement, string, flags=re.IGNORECASE)
    string += ' '
    if ' л ' in string:  # Удаляем пробелы перед "л"
        string = string.replace(' л ', '000 мл ')
        string = string.replace('.0', '')
    
    return string

In [ ]:
replace_values_l1('Кожный антисептик ПРОФ ДЗ "PROF DZ" готовый состав / 0,05 л')

In [ ]:
def string_filter_emb(string):
    
    string = string.lower() 
    string = string.replace('ф/п', ' ')
    string = re.sub(r'[^a-zo0-9а-я\s:]', ' ', string)
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    
    string = replace_values_kg(replace_values_l1(string))
    
    string = string.replace(' 0 ', ' ')
    string = ' '.join([w for w in string.split()])
    return string

In [ ]:
models = {'Infloat_multilingual_with_ozon_f1': DistanceRecommender(vectorizer=InfloatVectorizer(tokenizer=tokenizer, vectorizer=vectorizer), simularity_func=cosine_similarity, text_prep_func=string_filter_emb)}

In [ ]:
for model in models.values():
    model.fit(product, 'name_with_ozon', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name])])

In [ ]:
def remove_dots_except_between_numbers(string):
    new_string = ''
    for i, char in enumerate(string):
        if char in [',', '.']:
            if (string[i-1].isdigit() and
                i + 1 < len(string) and
                string[i+1].isdigit()):
                new_string += char
            else:
                new_string += ' '
                pass

        else:
            new_string += char
        previous_char = char

    return new_string

In [ ]:
def replace_values_2(string):
    res = []
    
    string = remove_dots_except_between_numbers(string)
    
    splitted = string.split()
    
    for i, t in enumerate(splitted):
        if 'л' in t:
            value = t.replace('л', '')
            if value.replace(',', '.').replace('.', '').isdigit():
                value = float(value.replace(',', '.')) * 1000
                t = f'{int(value)} мл'
                res += [t]
                continue

            elif splitted[i - 1].replace(',', '.').replace('.', '').isdigit() and not 'мл' in t:
                value = float(splitted[i - 1].replace(',', '.')) * 1000
                t = f'{int(value)} мл'
                res = res[:-1]
                res += [t]
                continue
            else:
                pass
        if 'кг' in t:
            value = t.replace('кг', '')
            if value.replace(',', '.').replace('.', '').isdigit():
                value = float(value.replace(',', '.')) * 1000
                t = f'{int(value)} г'
                res += [t]
                continue

            elif splitted[i - 1].replace(',', '.').replace('.', '').isdigit():
                value = float(splitted[i - 1].replace(',', '.')) * 1000
                t = f'{int(value)} г'
                res = res[:-1]
                res += [t]
                continue
            else:
                pass
        
        res += [t]
    return ' '.join(res)


        
            
    

In [ ]:
replace_values_2('Удалитель ржавчины PROSEPT RUST REMOVER, 500 мл, спрей')

In [ ]:
def string_filter_emb_1(string):
    
    string = string.lower() 
    string = replace_values_2(string)
    string = re.sub(r'[^a-zo0-9а-я\s:]', ' ', string)
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    
    string = string.replace(' 0 ', ' ')
    string = ' '.join([w for w in string.split()])
    return string

In [ ]:
models = {'Infloat_multilingual_with_ozon_f2.9': DistanceRecommender(vectorizer=InfloatVectorizer(tokenizer=tokenizer, vectorizer=vectorizer), simularity_func=cosine_similarity, text_prep_func=string_filter_emb_1)}

In [ ]:
for model in models.values():
    model.fit(product, 'name_with_ozon', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name])])

In [ ]:
results_table.sort_values('MRR', ascending=False)

In [ ]:
def string_filter_emb_2(string):
    
    string = string.lower() 
    string = replace_values_2(string)
    string = re.sub(r'[^a-zo0-9а-я\s:"]', ' ', string)
    string = re.sub(r'(?<=[а-я])(?=[a-z])|(?<=[a-z])(?=[а-я])', ' ', string)
    string = re.sub(r'(?<=[а-яa-z])(?=\d)|(?<=\d)(?=[а-яa-z])', ' ', string)
    
    string = string.replace(' 0 ', ' ')
    string = ' '.join([w for w in string.split()])
    return string

In [ ]:
models = {'Infloat_multilingual_with_ozon_f3': DistanceRecommender(vectorizer=InfloatVectorizer(tokenizer=tokenizer, vectorizer=vectorizer), simularity_func=cosine_similarity, text_prep_func=string_filter_emb_2)}

In [ ]:
for model in models.values():
    model.fit(product, 'name_with_ozon', 'id')

In [ ]:
for name, model in models.items():
    results = []
    acc_10 = []
    acc_5 = []
    acc_3 = []
    acc_1 = []
    
    for train_ind, test_ind in tqdm(kf.split(interactions)):

        test_interactions = interactions.loc[test_ind, :].merge(dealerprice,
                                                                left_on=['key', 'dealer_id'],
                                                                right_on=['product_key', 'dealer_id'],
                                                                how='inner')

        true_ids = test_interactions['product_id'].values

        recommendations = model.recommend(test_interactions['product_name'])
        
        results += [mean_reciprocal_rank(true_ids, recommendations)]
        acc_10 += [accuracy_k(true_ids, recommendations, 10)]
        acc_5 += [accuracy_k(true_ids, recommendations, 5)]
        acc_3 += [accuracy_k(true_ids, recommendations, 3)]
        acc_1 += [accuracy_k(true_ids, recommendations, 1)]
    
    results_table = pd.concat([results_table, pd.DataFrame({'MRR': np.mean(results),
                                                            'MRR_std': np.std(results),
                                                            'acc_10': np.mean(acc_10),
                                                            'acc_5': np.mean(acc_5),
                                                            'acc_3': np.mean(acc_3),
                                                            'acc_1': np.mean(acc_1)},
                                                           index=[name])])

In [ ]:
results_table.sort_values('MRR', ascending=False)